In [1]:
import shelve
import mkl_random as rnd
from tqdm import trange,tqdm
from numba import jit
import itertools

import pandas as pd
pd.options.compute.use_numba = False
import itertools
import math

from pgmpy.models import BayesianModel,BayesianNetwork,MarkovModel,NaiveBayes
from pgmpy.estimators import BayesianEstimator,ExhaustiveSearch,PC,HillClimbSearch,MaximumLikelihoodEstimator ,BicScore


from mysetup import NotebookFinder
import sys

sys.meta_path.append(NotebookFinder())

from setup_work import *
from msiit_db import MyDB
db = MyDB()

nominal = [x for x in db.eval_data.columns if "nominal_" in x]
ordinal = [x for x in db.eval_data.columns if "ordinal_" in x]
binary = [x for x in db.eval_data.columns if "binary_" in x]
interval = [x for x in db.eval_data.columns if "interval_" in x]
data_groups = [nominal,ordinal,binary,interval]

def gen_col_data(df):
    idx = df.index.to_flat_index().to_list()
    col_names = [f'col_{i}' for i in range(7)]
    df = pd.DataFrame(columns=col_names)
    for j,i_ in enumerate(idx):
        for k,c in zip(i_,col_names):
            df.loc[j,c] =k
    return df
orig_columns = ['binary__offer_expiration',
 'ordinal__income_range',
 'ordinal__no_visited_cold_drinks',
 'binary__travelled_more_than_15mins_for_offer',
 'ordinal__restaur_spend_less_than20',
 'nominal__marital_status',
 'nominal__restaurant_type',
 'ordinal__age',
 'binary__prefer_western_over_chinese',
 'binary__travelled_more_than_25mins_for_offer',
 'ordinal__no_visited_bars',
 'binary__gender',
 'nominal__car',
 'binary__restuarant_same_direction_house',
 'binary__cooks_regularly',
 'nominal__customer_type',
 'ordinal__qualif',
 'binary__is_foodie',
 'ordinal__no_take_aways',
 'nominal__job_industry',
 'binary__restuarant_opposite_direction_house',
 'binary__has_children',
 'ordinal__type_of_rest_rating',
 'interval__temperature',
 'ordinal__restaur_spend_greater_than20',
 'interval__travel_time',
 'interval__season',
 'ordinal__dest_distance',
 'binary__prefer_home_food',
 ]

master_X = pd.concat([db.data[db.eval_data.columns],db.eval_data],ignore_index=True,axis=0).astype(np.uint16)
enc = OrdinalEncoder()
enc.fit(master_X.loc[:,ordinal])
db.data[ordinal] = enc.transform(db.data[ordinal]).astype(np.uint16)
db.eval_data[ordinal] = enc.transform(db.eval_data.loc[:,ordinal]).astype(np.uint16)
db.nr_data[db.nr_eval_data.columns] = db.data[db.nr_eval_data.columns].astype(np.uint16)
db.nr_eval_data = db.eval_data[db.nr_eval_data.columns].astype(np.uint16)
master_X = pd.concat([db.data[db.eval_data.columns],db.eval_data],ignore_index=True,axis=0).astype(np.uint16)


importing Jupyter notebook from setup_work.ipynb


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


```python
data,eval_data = bring_data()

redundant,nr_data,nr_eval_data = elim_redund(data,eval_data)
```

In [2]:
pd.options.compute.use_numba=False
def check_independence(f,g):
    global db
    X = db.data
    y = db.y
    i=X.groupby(f)['target'].sum()/X.groupby(f)['target'].count()
    j=X.groupby(g)['target'].sum()/X.groupby(g)['target'].count()
    ij=X.groupby([f,g])['target'].sum()/X.groupby(f)['target'].count()

    proba = pd.DataFrame(columns=['(A,B)|C','A|C','B|C'],index=list(range(ij.shape[0])))
    for idx_,idx in enumerate(ij.index):
        a,b = idx
        proba.loc[idx_,:] = [i[a],j[b],ij.loc[a,b]]
    proba['P(A|C)*P(B|C)'] = proba.iloc[:,1]*proba.iloc[:,2]
    err = mean_squared_error(proba.iloc[0],proba.iloc[1],squared=False)
    return err
indpe = {x:{} for x in db.eval_data.columns}

for f1 in db.eval_data.columns:
    for f2 in db.eval_data.columns:
#         if f1==f2:
#             continue
#         else:
        indpe[f1][f2] = check_independence(f1,f2)

In [3]:
indp: pd.DataFrame = pd.DataFrame(indpe)
indp_ = indp.where(indp<0.001,other=0)
indp_['all_sum'] = indp_.sum(axis=1)
v  = indp_[indp_['all_sum']>0].index
# indp.loc[v,:]


In [4]:
X = db.data[db.orig_columns+['target']]
# model = BayesianModel(feature_vars=db.orig_columns,dependent_var='target')
model = BayesianModel()
model.fit(X,estimator=MaximumLikelihoodEstimator)

In [9]:
model = HillClimbSearch(X)
mod = model.estimate(scoring_method=BicScore(X))

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [5]:
freq = X.groupby('ordinal__income_range')['ordinal__restaur_spend_greater_than20'].value_counts()
low =( freq[0]+ freq[1]+ freq[2]).sort_index() /4765
mid = (freq[3]+ freq[4]+ freq[5]).sort_index()/4098
hi = (freq[6]+ freq[7]+ freq[8]).sort_index()/3416
low,mid,hi
spend_pattern = pd.DataFrame(columns=["low","mid","hi"])
spend_pattern.low = low
spend_pattern.mid = mid
spend_pattern.hi = hi
spend_pattern_g20 = spend_pattern.copy()


freq = X.groupby('ordinal__income_range')['ordinal__restaur_spend_less_than20'].value_counts()
low =( freq[0]+ freq[1]+ freq[2]).sort_index() /4765
mid = (freq[3]+ freq[4]+ freq[5]).sort_index()/4098
hi = (freq[6]+ freq[7]+ freq[8]).sort_index()/3416
low,mid,hi
spend_pattern = pd.DataFrame(columns=["low","mid","hi"])
spend_pattern.low = low
spend_pattern.mid = mid
spend_pattern.hi = hi
spend_pattern_l20 = spend_pattern.copy()

spend_pattern_g20.loc[2:,:].sum(),spend_pattern_l20.loc[3:,:].sum()

master_X = pd.concat([db.data[db.eval_data.columns],db.eval_data],ignore_index=True,axis=0)

In [ ]:
a = 'interval__travel_time'
b = 'ordinal__qualif'
freq = X.groupby(["interval__season",'ordinal__dest_distance','interval__temperature','interval__travel_time',])['target'].mean()
# freq

# freq[1][1].unstack().div(freq[1][1].unstack().sum(axis=1),axis=0)



# _ = db.data[db.orig_columns].plot(kind='hist',subplots=True,figsize=(30,20),layout=(6,5),sharex=False,sharey=False,bins=20)

# n_uniq = db.data.nunique()
# for c in db.orig_columns:
#     if n_uniq[c]>2 and n_uniq[c]<6:
#         fig,ax= plt.subplots(1,1,)
#         _ = db.data[c].plot(kind='hist',ax=ax)
#         _ = plt.legend()

db.data.groupby(['ordinal__income_range'])['ordinal__restaur_spend_greater_than20'].value_counts().sort_index()

In [7]:
# X = pd.get_dummies(X,columns=[x for x in db.orig_columns if x!='target' and 'ordinal_' not in x and 'binary__' not in x])
[x for x in db.orig_columns if x!='target' and 'ordinal_' not in x and 'binary__' not in x]

['nominal__marital_status',
 'nominal__restaurant_type',
 'nominal__car',
 'nominal__customer_type',
 'nominal__job_industry',
 'interval__temperature',
 'interval__travel_time',
 'interval__season']

In [115]:
ordinal

['ordinal__income_range',
 'ordinal__no_visited_cold_drinks',
 'ordinal__restaur_spend_less_than20',
 'ordinal__age',
 'ordinal__no_visited_bars',
 'ordinal__qualif',
 'ordinal__no_take_aways',
 'ordinal__type_of_rest_rating',
 'ordinal__restaur_spend_greater_than20',
 'ordinal__dest_distance']

In [2]:
X = db.data[db.orig_columns]
X.columns
X_eval= db.eval_data[db.orig_columns]
X_eval['dum'] = 0


In [ ]:
inve = X.groupby(['interval__season','interval__temperature', 'interval__travel_time'])['target'].sum()/X.groupby(['interval__season','interval__temperature','interval__travel_time'])['target'].count()
# X.groupby(['interval__season','interval__temperature'])['target'].value_counts()
inve
piv = ['interval__season','ordinal__dest_distance','interval__temperature']
X.groupby(piv)['target'].count()/X_eval.groupby(piv)['dum'].count()


In [5]:
nominal = [x for x in db.orig_columns if "nominal_" in x]
ordinal = [x for x in db.orig_columns if "ordinal_" in x]
binary = [x for x in db.orig_columns if "binary_" in x]
interval = [x for x in db.orig_columns if "interval_" in x]
# nominal = [x for x in db.nr_eval_data.columns if "nominal_" in x]
# ordinal = [x for x in db.nr_eval_data.columns if "ordinal_" in x]
# binary = [x for x in db.nr_eval_data.columns if "binary_" in x]
# interval = [x for x in db.nr_eval_data.columns if "interval_" in x]
X = db.data[db.orig_columns]
X[nominal+['interval__season']]  = X[nominal+['interval__season']].astype('category')
X = X.drop(['nominal__car'],axis=1)
# X.info()

In [6]:
X

,binary__offer_expiration,ordinal__income_range,ordinal__no_visited_cold_drinks,binary__travelled_more_than_15mins_for_offer,ordinal__restaur_spend_less_than20,nominal__marital_status,nominal__restaurant_type,ordinal__age,binary__prefer_western_over_chinese,binary__travelled_more_than_25mins_for_offer,ordinal__no_visited_bars,binary__gender,binary__restuarant_same_direction_house,binary__cooks_regularly,nominal__customer_type,ordinal__qualif,binary__is_foodie,ordinal__no_take_aways,nominal__job_industry,binary__restuarant_opposite_direction_house,binary__has_children,ordinal__type_of_rest_rating,interval__temperature,ordinal__restaur_spend_greater_than20,interval__travel_time,interval__season,ordinal__dest_distance,binary__prefer_home_food
0,1,8.0,3.0,1,1.0,0,0,4.0,0,0,1.0,1,0,1,0,4.0,0,2.0,0,0,0,4.0,67,1.0,22,1,2.0,0
1,1,7.0,3.0,0,3.0,0,1,7.0,0,0,0.0,1,1,1,0,4.0,0,2.0,0,0,1,3.0,89,2.0,18,2,2.0,0
2,1,7.0,1.0,1,2.0,1,2,2.0,1,0,0.0,1,1,0,0,4.0,1,2.0,1,1,1,4.0,67,1.0,7,3,1.0,1
3,0,3.0,1.0,0,2.0,1,1,6.0,1,0,0.0,0,1,0,0,2.0,1,2.0,2,0,1,3.0,89,2.0,7,2,0.0,0
4,1,8.0,0.0,1,2.0,1,0,1.0,0,1,1.0,1,0,0,0,2.0,1,4.0,2,1,0,3.0,40,1.0,7,2,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12374,0,2.0,1.0,1,4.0,3,3,7.0,0,0,0.0,1,0,0,0,2.0,1,1.0,2,1,1,1.0,89,0.0,22,2,2.0,0
12375,1,2.0,3.0,0,0.0,1,1,1.0,0,0,2.0,0,0,0,1,2.0,1,1.0,22,1,1,2.0,67,0.0,18,2,2.0,0
12376,0,7.0,3.0,1,4.0,0,2,3.0,0,1,2.0,1,0,1,0,4.0,1,2.0,7,1,0,2.0,89,4.0,18,1,2.0,1
12377,0,1.0,3.0,1,1.0,0,4,2.0,0,0,0.0,0,0,1,0,4.0,0,3.0,3,1,0,1.0,89,1.0,7,2,0.0,0


In [14]:
# X = db.data[db.orig_columns].astype('category')
import xgboost as xgb
y = db.y
X_train, X_test, y_train, y_test = gen_train_test(X,y,test_size=0.2)
clf = xgb.XGBClassifier(
    tree_method='approx',
    enable_categorical=True,
    max_cat_to_onehot=90,
    booster='dart',
    grow_policy='lossguide',
    eval_metric='aucpr',
    max_depth=5,
    importance_type='total_cover',
    num_parallel_tree=3,
#     n_estimators=1000,
    random_state=42,
    n_jobs=-1
                    
                
                   )
with parallel_backend('loky'):
    y_pred=clf.fit(X_train,y_train).predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.58      0.52      0.55      1077
           1       0.66      0.72      0.69      1399

    accuracy                           0.63      2476
   macro avg       0.62      0.62      0.62      2476
weighted avg       0.63      0.63      0.63      2476



In [100]:
X = db.data[db.orig_columns].astype('category')
y = db.y
X_train, X_test, y_train, y_test = gen_train_test(X,y,test_size=0.2)
clf = XGBClassifier(tree_method='hist',enable_categorical=True,max_cat_to_onehot=90,)
y_pred=clf.fit(X_train,y_train).predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.58      0.52      0.55      1077
           1       0.66      0.71      0.68      1399

    accuracy                           0.63      2476
   macro avg       0.62      0.62      0.62      2476
weighted avg       0.62      0.63      0.62      2476



In [ ]:
clf.feature_importances_
fi = {x:v for x,v in zip(db.orig_columns,clf.feature_importances_)}
sorted(fi,key=fi.get,reverse=True),fi

In [129]:
X = db.data[db.orig_columns]
y = db.y
X_eval = db.eval_data[db.orig_columns]
train_data_sets = {}
eval_data_sets = {}
g = 'interval__season'
h = 'interval__temperature'
f = 'ordinal__dest_distance'
ug = [1, 2, 3]
uf = [0, 1, 2]
uh = [67, 89, 40]
other_cols = np.setdiff1d(db.orig_columns,[f,h,g])
d_idx = 0
for i in uf:
    for j in ug:
        for k in uh:
            train_idx = X[X[f]==i][X[g]==j][X[h]==k].index
            eval_idx = X_eval[X_eval[f]==i][X_eval[g]==j][X_eval[h]==k].index
            train_data_sets[d_idx] = (X.loc[train_idx,other_cols],y.loc[train_idx])
            eval_data_sets[d_idx] = X_eval.loc[eval_idx,other_cols]
            d_idx += 1
master_data_sets = {}
for pr in range(27):
    X_,y_ = train_data_sets[pr]
    master_data_sets[pr] = pd.concat([X_,eval_data_sets[pr]],ignore_index=True,axis=0)
master_data_sets[0]

,binary__cooks_regularly,binary__gender,binary__has_children,binary__is_foodie,binary__offer_expiration,binary__prefer_home_food,binary__prefer_western_over_chinese,binary__restuarant_opposite_direction_house,binary__restuarant_same_direction_house,binary__travelled_more_than_15mins_for_offer,binary__travelled_more_than_25mins_for_offer,interval__travel_time,nominal__car,nominal__customer_type,nominal__job_industry,nominal__marital_status,nominal__restaurant_type,ordinal__age,ordinal__income_range,ordinal__no_take_aways,ordinal__no_visited_bars,ordinal__no_visited_cold_drinks,ordinal__qualif,ordinal__restaur_spend_greater_than20,ordinal__restaur_spend_less_than20,ordinal__type_of_rest_rating
0,1,0,0,0,1,0,1,0,1,1,0,7,0,1,2,1,1,2,6,3,0,0,4,1,3,2
1,0,1,0,1,1,0,0,0,1,0,0,7,0,0,5,0,2,3,8,4,1,1,2,1,3,4
2,1,0,0,0,1,1,1,1,0,0,0,7,0,0,18,1,4,1,4,1,2,0,4,0,3,4
3,0,1,1,0,1,0,0,1,0,1,1,7,0,0,13,0,0,7,4,1,0,0,4,1,1,1
4,0,0,1,1,0,0,1,1,0,1,0,18,0,1,1,0,2,5,2,4,0,2,2,1,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,0,1,1,1,1,1,0,0,1,0,0,7,0,2,19,2,2,7,3,3,2,3,2,3,4,0
222,0,0,0,0,0,0,0,0,0,1,0,14,0,3,3,1,1,1,8,1,3,0,4,0,0,2
223,1,0,0,1,1,1,0,1,0,1,0,10,0,3,0,1,0,2,0,2,1,1,3,1,1,4
224,1,1,0,1,1,1,0,1,0,1,0,18,0,3,7,0,0,2,2,2,3,1,4,2,2,3


In [126]:
g = 'interval__season'
h = 'interval__temperature'
f = 'ordinal__dest_distance'
ug = [1, 2, 3]
uf = [0, 1, 2]
uh = [67, 89, 40]
other_cols = np.setdiff1d(db.orig_columns,[f,h,g])
d_idx = 0
for i in uf:
    for j in ug:
        for k in uh:
            train_idx = X[X[f]==i][X[g]==j][X[h]==k].index
            eval_idx = X_eval[X_eval[f]==i][X_eval[g]==j][X_eval[h]==k].index
            train_data_sets[d_idx] = (X.loc[train_idx,other_cols],y.loc[train_idx])
            eval_data_sets[d_idx] = X_eval.loc[eval_idx,other_cols]
            d_idx += 1
#             print(len(train_idx)/len(eval_idx),ert)


In [111]:
X[f].unique()

array([0, 1, 2, 3])

In [36]:
ct = make_column_transformer(
    (Normalizer(),make_column_selector(pattern='interval__*|ordinal_*')),
    (OneHotEncoder(sparse=True),make_column_selector(pattern='nominal_*|interval__*|ordinal__*')),
    remainder='passthrough',
    sparse_threshold=50,
    n_jobs=-1
)
enc = make_pipeline(ct,PolynomialFeatures(include_bias=False,interaction_only=True,degree=3,order='F'),VarianceThreshold())
enc

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(n_jobs=-1, remainder='passthrough',
                                   sparse_threshold=50,
                                   transformers=[('normalizer', Normalizer(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f4439bd29a0>),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f4439bd2700>)])),
                ('polynomialfeatures',
                 PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True, order='F')),
                ('variancethreshold', VarianceThreshold())])

In [37]:
with parallel_backend('loky',):
    enc.fit(master_X[db.orig_columns])
    X = db.data[db.orig_columns]
    X_enc = enc.transform(X)
X_enc = pd.DataFrame.sparse.from_spmatrix(X_enc,columns=list(enc.get_feature_names_out()))    

In [22]:
import gc;gc.collect()

0

<html><head>


<!-- Load require.js. Delete this if your page already loads require.js -->
<script src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.4/require.min.js" integrity="sha256-Ae2Vz/4ePdIu6ZyI/5ZGsYnb+m0JlOmKPjt6XZ9JJkA=" crossorigin="anonymous"></script>
<script src="https://unpkg.com/@jupyter-widgets/html-manager@*/dist/embed-amd.js" crossorigin="anonymous"></script>
<script type="application/vnd.jupyter.widget-state+json">
{
    "version_major": 2,
    "version_minor": 0,
    "state": {}
}
</script>
</head>
<body>


</body>
</html>


In [39]:
X_enc.memory_usage().sum()

1050400448

In [91]:
X_enc = X_enc.astype(np.uint16)

In [ ]:
(12379, 310247)

In [38]:
X_enc.shape

(12379, 309077)

In [41]:
clf = RidgeClassifierCV(scoring='f1_macro',fit_intercept=False,class_weight='balanced',cv=RepeatedStratifiedKFold())
y = db.y
X_train, X_test, y_train, y_test = gen_train_test(X_enc,y,test_size=0.2)
with parallel_backend('loky',n_jobs=-1):
    y_pred=clf.fit(X_train,y_train).predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.52      0.52      0.52      1077
           1       0.63      0.63      0.63      1399

    accuracy                           0.59      2476
   macro avg       0.58      0.58      0.58      2476
weighted avg       0.59      0.59      0.59      2476



In [45]:
X_val_enc = enc.transform(db.eval_data[db.orig_columns])
X_val_enc= pd.DataFrame.sparse.from_spmatrix(X_val_enc,columns=list(enc.get_feature_names_out()))    

In [46]:
X_val_enc.to_pickle(DATA_OUTPUT+'pold_eval.parquet')